In [7]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
from sklearn.model_selection import train_test_split

# Configuración
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 64
EPOCHS = 5

# Rutas corregidas del dataset
BASE_PATH = '/kaggle/input/human-bone-fractures-image-dataset/Human Bone Fractures Multi-modal Image Dataset (HBFMID)/Bone Fractures Detection'
TRAIN_IMG_PATH = os.path.join(BASE_PATH, 'train/images')
TRAIN_LABEL_PATH = os.path.join(BASE_PATH, 'train/labels')
VALID_IMG_PATH = os.path.join(BASE_PATH, 'valid/images')
VALID_LABEL_PATH = os.path.join(BASE_PATH, 'valid/labels')
TEST_IMG_PATH = os.path.join(BASE_PATH, 'test/images')
TEST_LABEL_PATH = os.path.join(BASE_PATH, 'test/labels')

In [8]:
def load_images_and_labels(img_path, label_path):
    images = []
    labels = []
    
    image_files = sorted([f for f in os.listdir(img_path) if f.endswith(('.jpg', '.png', '.jpeg'))])
    
    for img_file in image_files:
        img_full_path = os.path.join(img_path, img_file)
        label_file = os.path.splitext(img_file)[0] + '.txt'
        label_full_path = os.path.join(label_path, label_file)
        
        img = load_img(img_full_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
        img_array = img_to_array(img) / 255.0
        images.append(img_array)
        
        # Si existe archivo de label, hay fractura (1), sino no hay (0)
        if os.path.exists(label_full_path) and os.path.getsize(label_full_path) > 0:
            labels.append(1)
        else:
            labels.append(0)
    
    return np.array(images), np.array(labels)

print("Cargando datos de entrenamiento...")
X_train, y_train = load_images_and_labels(TRAIN_IMG_PATH, TRAIN_LABEL_PATH)
print(f"Train: {X_train.shape[0]} imagenes")

print("Cargando datos de validacion...")
X_valid, y_valid = load_images_and_labels(VALID_IMG_PATH, VALID_LABEL_PATH)
print(f"Valid: {X_valid.shape[0]} imagenes")

print("Cargando datos de prueba...")
X_test, y_test = load_images_and_labels(TEST_IMG_PATH, TEST_LABEL_PATH)
print(f"Test: {X_test.shape[0]} imagenes")

Cargando datos de entrenamiento...
Train: 1347 imagenes
Cargando datos de validacion...
Valid: 128 imagenes
Cargando datos de prueba...
Test: 64 imagenes


In [12]:
def create_cnn_model():
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.5),
        
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.5),
        
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.5),
        
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    
    return model

model = create_cnn_model()
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("Arquitectura del modelo:")
model.summary()

Arquitectura del modelo:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 24, 24, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     2,359,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,600,385 (9.92 MB)

 Trainable params: 2,600,385 (9.92 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
history = model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_valid, y_valid),
    verbose=1
)

print("\nEntrenamiento completado")

Epoch 1/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 272ms/step - accuracy: 0.8806 - loss: 0.1808 - val_accuracy: 1.0000 - val_loss: 0.0608
Epoch 2/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - accuracy: 0.9992 - loss: 0.0462 - val_accuracy: 1.0000 - val_loss: 0.1934
Epoch 3/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.9990 - loss: 0.0269 - val_accuracy: 1.0000 - val_loss: 0.0325
Epoch 4/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.9978 - loss: 0.0141 - val_accuracy: 1.0000 - val_loss: 0.0036
Epoch 5/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.9988 - loss: 0.0191 - val_accuracy: 1.0000 - val_loss: 0.0520

Entrenamiento completado


In [14]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nResultados en Test:")
print(f"Loss: {test_loss:.4f}")
print(f"Accuracy: {test_accuracy:.4f}")

# Predicciones
y_pred = (model.predict(X_test, verbose=0) > 0.5).astype(int)
print(f"\nTotal de predicciones: {len(y_pred)}")
print(f"Fracturas detectadas: {np.sum(y_pred)}")
print(f"Sin fractura: {len(y_pred) - np.sum(y_pred)}")


Resultados en Test:
Loss: 0.0530
Accuracy: 1.0000

Total de predicciones: 64
Fracturas detectadas: 64
Sin fractura: 0
